<a href="https://colab.research.google.com/github/pratim808/hhealth-motor-insurance-chatbot-using-ai-agents/blob/main/insurance_ai_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -U langsmith langchain langchain-community  sentence-transformers langchain_huggingface faiss-gpu langchain_groq

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents import AgentExecutor

# Initialize embeddings (only need to do this once)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# --- Health Insurance Setup ---
health_loader = WebBaseLoader("https://www.tataaig.com/health-insurance")
health_docs = health_loader.load()
health_documents = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=500
).split_documents(health_docs)
health_vector = FAISS.from_documents(health_documents, embeddings)
health_retriever = health_vector.as_retriever(search_kwargs={"k": 3})
health_tool = create_retriever_tool(
    health_retriever,
    "health_insurance_search",
    "Search for information about Health Insurance. For any questions about Health Insurance, you must use this tool!",
)

# --- Motor Insurance Setup ---
motor_loader = WebBaseLoader("https://www.tataaig.com/motor-insurance")
motor_docs = motor_loader.load()
motor_documents = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=500
).split_documents(motor_docs)
motor_vector = FAISS.from_documents(motor_documents, embeddings)
motor_retriever = motor_vector.as_retriever(search_kwargs={"k": 3})
motor_tool = create_retriever_tool(
    motor_retriever,
    "motor_insurance_search",
    "Search for information about Motor Insurance. For any questions about Motor Insurance, you must use this tool!",
)

# --- LLM Setup ---
groq_api_key = ('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')
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

# --- Tools ---
tools = [health_tool, motor_tool]



In [ ]:
# --- Prompt ---
'''prompt = hub.pull("hwchase17/react-chat")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)'''
prompt = hub.pull("hwchase17/react-chat")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
    instructions="For any question about health insurance, you must use the `health_insurance_search` tool. For any question about motor insurance, you must use the `motor_insurance_search` tool."
)
llm_with_stop = llm.bind(stop=["\nObservation"])




In [ ]:
# --- Agent ---
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)
def handle_agent_error(error, **kwargs):
    print(f"Agent Error: {error}")
    return {"output": "I encountered an error. Please try again."}

memory = ConversationBufferMemory(memory_key="chat_history")

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,memory=memory, handle_parsing_errors=handle_agent_error)


In [ ]:
print(agent_executor.invoke({"input": "What is the Waiting Period in a Medical Insurance Policy?"})["output"])

## Analogy

Imagine you're trying to answer a question about a specific book.

Thought: "Do I need to use a tool? Yes" - You realize you need to look up the book.

Action: You go to your bookshelf and find the book.

Observation: You read the relevant section of the book.

Thought: "Do I need to use a tool? No" - You have the information you need.

Final Answer: You answer the question.

In [ ]:
print(agent_executor.invoke({"input": "Tell me about motor insurance plans?"})["output"])

In [ ]:
print(agent_executor.invoke({"input": "Tell me about health insurance plans?"})["output"])

In [ ]:
print(agent_executor.invoke({"input": "What is the process for filing a claim for car damage?"})["output"])

In [ ]:
print(agent_executor.invoke({"input": "Why Should I Buy a Vehicle Insurance Policy?"})["output"])

In [ ]:
Why Should I Buy a Vehicle Insurance Policy?
Types of Motor Insurance

In [ ]:
print(agent_executor.invoke({"input": "Types of Motor Insurance"})["output"])

In [ ]:
Comprehensive Motor Insurance Policy

In [ ]:
print(agent_executor.invoke({"input": "what is Comprehensive Motor Insurance Policy"})["output"])

## Original Answer
A comprehensive vehicle insurance policy offers extensive coverage as compared to a basic third-party motor insurance policy.
The comprehensive motor insurance policy offers protection against financial losses due to damage to your own car and third-party liabilities.

## Another way to get the response

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [ ]:
conversational_agent.run({"input" :  "What’s Not Covered in TATA AIG Motor Insurance?"})

In [ ]:
print(agent_executor.invoke({"input": "what is Comprehensive Motor Insurance Policy"})["output"])